## Fully Connected layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
t_max = 2**8
batch_size = 2
dims = (2,10,10)
n_outputs = 2

np.random.seed(seed=24)
np.set_printoptions(suppress=True)
weights = (np.random.rand(n_outputs,np.product(dims)) - 0.5) / 4 # np.ones((n_outputs,np.product(dims))) # 
biases = (np.random.rand(n_outputs) - 0.5) / 2 # np.zeros((n_outputs)) # 

loihi_model = quartz.Network([
    layers.InputLayer(dims=dims),
    layers.Dense(weights=weights, biases=biases),
    layers.MonitorLayer(),
])

# input0 = quartz.probe(loihi_model.layers[0].blocks[-1])
# relcos = [quartz.probe(block) for block in loihi_model.layers[1].blocks[::3]]
# calcs = [quartz.probe(block.neurons[0]) for block in loihi_model.layers[1].blocks[::3]]
# splitters = [quartz.probe(block) for block in loihi_model.layers[1].blocks[2::3]]

values = np.random.rand(batch_size,*dims) / 2 # np.ones((np.product(dims))) * 0.5 # np.zeros((np.product(dims))) # 
quantized_values = (values*t_max).round()/t_max
weight_acc = loihi_model.layers[1].weight_acc
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

pt_model = nn.Sequential(
    nn.Linear(in_features=np.product(dims), out_features=n_outputs), 
    nn.ReLU()
)
pt_model[0].weight = torch.nn.Parameter(torch.tensor(quantized_weights))
pt_model[0].bias = torch.nn.Parameter(torch.tensor((quantized_biases)))
model_output = pt_model(torch.tensor(quantized_values.reshape(batch_size, -1))).detach().numpy()
model_output

In [ ]:
loihi_output = loihi_model(values, t_max, steps_per_image=1600) #, partition='nahuku32', profiling=True)

In [ ]:
loihi_output

In [ ]:
ok = [relco.plot() for relco in relcos]

In [ ]:
ok = [calc.plot() for calc in calcs]

## Convolutional Layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

t_max = 2**8
input_dims = (2,4,4)
weight_dims = (4,2,2,2)
batch_size = 1
kernel_size = weight_dims[2:]
np.random.seed(seed=27)
weights = (np.random.rand(*weight_dims)-0.5) / 3 # np.ones((weight_dims)) # 
biases = (np.random.rand(weight_dims[0])-0.5) / 2 # np.zeros((weight_dims[0])) #

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights, biases=biases),
    layers.MonitorLayer(),
])

input0 = quartz.probe(loihi_model.layers[0].blocks[0])
relcos = [quartz.probe(block) for block in loihi_model.layers[1].blocks]

values = np.random.rand(batch_size, *input_dims) / 2 # np.zeros((np.product(input_dims)))
quantized_values = (values*t_max).round()/t_max
weight_acc = loihi_model.layers[1].weight_acc
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=kernel_size), nn.ReLU())
model[0].weight = torch.nn.Parameter(torch.tensor(quantized_weights))
model[0].bias = torch.nn.Parameter(torch.tensor((quantized_biases)))
model_output = model(torch.tensor(quantized_values)).detach().numpy()
#model_output

In [ ]:
loihi_output = loihi_model(values, t_max, steps_per_image=1600)
loihi_output[0]

In [ ]:
ok = [relco.plot() for relco in relcos]

## Max Pooling layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

t_max = 2**8
batch_size = 2
input_dims = (1,10,10)
kernel_size = [2,2]

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.MaxPool2D(kernel_size=kernel_size),
    layers.MonitorLayer(),
])

pool0 = quartz.probe(loihi_model.layers[1].blocks[0])

np.random.seed(seed=45)
values = np.random.rand(batch_size, *input_dims)
quantized_values = (values*t_max).round()/t_max

model = nn.Sequential(nn.MaxPool2d(kernel_size=kernel_size, stride=kernel_size[0]), nn.ReLU())
model_output = model(torch.tensor(quantized_values)).squeeze().detach().numpy()
model_output

In [ ]:
loihi_model(values, t_max, steps_per_image=2600)

## Convpool layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = ( 2,4,4)
weight_dims = ( 5,2,3,3)
t_max = 2**8
batch_size = 2
conv_kernel_size = weight_dims[2:]
pooling_kernel_size = [2,2]
pooling_stride = 2

np.random.seed(seed=47)
np.set_printoptions(suppress=True)
weights = (np.random.rand(*weight_dims)-0.5) / 2 # np.zeros(weight_dims) #
biases = (np.random.rand(weight_dims[0])-0.5) # np.zeros(weight_dims[0]) #

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.ConvPool2D(weights=weights, biases=biases, pool_kernel_size=pooling_kernel_size),
    layers.MonitorLayer(),
])
trigger0 = quartz.probe(loihi_model.layers[0].blocks[-1])
trigger1 = quartz.probe(loihi_model.layers[1].blocks[0])
convmaxs1 = [quartz.probe(block) for block in loihi_model.layers[1].blocks[11:]]

loihi_model1 = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights, biases=biases),
    layers.MaxPool2D(kernel_size=pooling_kernel_size, stride=pooling_stride),
    layers.MonitorLayer(),
])
values = np.random.rand(batch_size, *input_dims)
quantized_values = (values*t_max).round()/t_max
weight_acc = loihi_model.layers[1].weight_acc
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(weights))
model[0].bias = nn.Parameter(torch.tensor(biases))
model_output = model(torch.tensor(values)).detach().numpy()
model_output

In [ ]:
loihi_output = loihi_model(values, t_max, steps_per_image=2600)
loihi_output

In [ ]:
ok = [convmax.plot() for convmax in convmaxs1]